In [13]:
import numpy as np
from keras.models import Model # Импортируем модели keras: Model
# Импортируем стандартные слои keras
from keras.layers import Input, Conv2DTranspose, concatenate, Activation
from keras.layers import MaxPooling2D, Conv2D 
from keras.models import load_model
from keras.optimizers import Adam  # Импортируем оптимизатор Adam
from PIL import Image

In [14]:
def PSPnet_7(num_classes = 3, input_shape= (88, 120, 3)):
    img_input = Input(input_shape) 
    #print(K.int_shape(img_input))

    x = Conv2D(28, (3, 3), padding='same')(img_input)
    x = Activation(function_activation)(x)
    x = Conv2D(28, (3, 3), padding='same')(x)
    x = Activation(function_activation)(x)
    x = Conv2D(28, (3, 3), padding='same')(x)
    x = Activation(function_activation)(x)

    x = Conv2D(56, (3, 3), padding='same')(x)
    x = Activation(function_activation)(x)
    x = Conv2D(56, (3, 3), padding='same')(x)
    x = Activation(function_activation)(x)
    x = Conv2D(56, (3, 3), padding='same')(x)
    x = Activation(function_activation)(x)

    x = Conv2D(224, (3, 3), padding='same')(x)
    x = Activation(function_activation)(x)
    x = Conv2D(224, (3, 3), padding='same')(x)
    x = Activation(function_activation)(x)
    x = Conv2D(224, (3, 3), padding='same')(x)
    x = Activation(function_activation)(x)

    # pooling пирамида
    block_1_out = MaxPooling2D((2, 2))(x)
    block_2_out = MaxPooling2D((4, 4))(x)
    block_3_out = MaxPooling2D((8, 8))(x)
    block_4_out = MaxPooling2D((16, 16))(x)

    # Сверточные слои
    block_1_out = Conv2D(56, (3, 3), padding='same', name='block1_conv1')(block_1_out)
    block_1_out = Activation(function_activation)(block_1_out)

    block_2_out = Conv2D(56, (3, 3), padding='same', name='block2_conv1')(block_2_out)
    block_2_out = Activation(function_activation)(block_2_out)

    block_3_out = Conv2D(56, (3, 3), padding='same', name='block3_conv1')(block_3_out)
    block_3_out = Activation(function_activation)(block_3_out)

    block_4_out = Conv2D(56, (3, 3), padding='same', name='block4_conv1')(block_4_out)
    block_4_out = Activation(function_activation)(block_4_out)

    # Увеличиваем размерность
    block_1_out = Conv2DTranspose(28, (3, 3), strides=(2, 2), padding='same')(block_1_out)
    block_1_out = Activation(function_activation)(block_1_out)

    block_2_out = Conv2DTranspose(28, (3, 3), strides=(4, 4), padding='same')(block_2_out)
    block_2_out = Activation('relu')(block_2_out)

    block_3_out = Conv2DTranspose(28, (3, 3), strides=(8, 8), padding='same')(block_3_out)
    block_3_out = Activation(function_activation)(block_3_out)

    block_4_out = Conv2DTranspose(28, (3, 3), strides=(16, 16), padding='same')(block_4_out)
    block_4_out = Activation(function_activation)(block_4_out)

    #объединяем слои
    x = concatenate([block_1_out, block_2_out, block_3_out, block_4_out])

    x = Conv2D(224, (3, 3), padding='same')(x)
    x = Activation(function_activation)(x)
    x = Conv2D(224, (3, 3), padding='same')(x)
    x = Activation(function_activation)(x)
    x = Conv2D(224, (3, 3), padding='same')(x)
    x = Activation(function_activation)(x)

    x = Conv2D(56, (3, 3), padding='same')(x)
    x = Activation(function_activation)(x)
    x = Conv2D(56, (3, 3), padding='same')(x)
    x = Activation(function_activation)(x)
    x = Conv2D(56, (3, 3), padding='same')(x)
    x = Activation(function_activation)(x)

    x = Conv2D(28, (3, 3), padding='same')(x)
    x = Activation(function_activation)(x)
    x = Conv2D(28, (3, 3), padding='same')(x)
    x = Activation(function_activation)(x)
    x = Conv2D(28, (3, 3), padding='same')(x)
    x = Activation(function_activation)(x)
    
    out = Conv2D(num_classes, (3, 3), activation = function_out_activation, padding = 'same')(x)
    # Выходной слой

    model = Model(img_input, out) # Создаем модель с входом 'img_input' и выходом 'out'
    
    return model # Возвращаем сформированную модель

In [15]:
img_width = 176 # Ширина уменьшенной картинки 
img_height = 240 # Высота уменьшенной картинки 
img_cannels = 3 # Количество каналов 3
num_classes = 7 # Задаем количество классов на изображении
start_lr = 0.00001

function_activation = 'relu'
function_out_activation = 'softmax'

# Компилируем модель 
model = PSPnet_7(num_classes, (img_width, img_height, img_cannels)) # Создаем модель unet
model.compile(optimizer = Adam(start_lr),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
print('Я скомпилировалась')

Я скомпилировалась


In [16]:
model.load_weights('12Pro1_model_7_PSPnet_16_100.h5')

model.save('Test_prototype_model.h5')

In [17]:
# Функция преобразования индекса в цвет пикселя
def index2color(index2):
  index = np.argmax(index2) # Получаем индекс максимального элемента
  color=[]
  if index == 0: color = [100, 100, 100]  # пол
  elif index == 1: color = [0, 0, 100]    # потолок
  elif index == 2: color = [0, 100, 0]    # стена
  elif index == 3: color = [100, 0, 0]    # проем, дверь, окно
  elif index == 4: color = [0, 100, 100]  # колонна, лестница, внешний мир, перила, батарея, инвентарь, источники света, провода, балка
  elif index == 5: color = [100, 0, 100]  # люди
  elif index == 6: color = [0, 0, 0]      # остальное
  return color # Возвращаем цвет пикслея

In [18]:
input_image = Image.open("Img_901.jpg").resize((img_height, img_width))
input_array = np.array(input_image, dtype = 'float64') / 255
print('input_array', input_array.shape)

output_array = np.array(model.predict(input_array.reshape((1, ) + input_array.shape)))
print('output_array', output_array.shape)

pr = output_array.reshape(-1, num_classes) # Решейпим предикт
print('pr', pr.shape)

pr1 = [] # Пустой лист под сегментированную картинку из predicta
for k in range(len(pr)): # Проходим по всем уровням (количесвто классов)
    pr1.append(index2color(pr[k])) # Переводим индекс в писксель

pr1 = np.array(pr1) # Преобразуем в numpy
print('pr1', pr1.shape)

pr1 = pr1.reshape(img_width, img_height, 3) # Решейпим к размеру изображения
print('reshape pr1', pr1.shape)

img = Image.fromarray(pr1.astype('uint8')) # Получаем картику из предикта

img.save("OutImage.jpg")

input_array (176, 240, 3)
1/1 [==============================] - 4s 4s/step
output_array (1, 176, 240, 7)
pr (42240, 7)
pr1 (42240, 3)
reshape pr1 (176, 240, 3)
